# PHISHING WEBSITE DETECTION

Importing the Required Packages

In [1]:
# packages
import pandas as pd
from urllib.parse import urlparse,urlencode
import re
from bs4 import BeautifulSoup
import requests
#import whois
import urllib.request
from datetime import datetime
import time
import socket
from urllib.error import HTTPError
from cython.parallel import prange

Importing the Text file Containing Phishing Website url's 

In [2]:
computer_raw_data=pd.read_csv("C:\\Users\\gouru\\Downloads\\Phishing-Website-Detection-master\\Phishing-Website-Detection-master\\raw_datasets\\1000-phishing.txt",header=None,names=['urls'])
computer_raw_data.head()

,urls
0,http://asesoresvelfit.com/media/datacredito.co/
1,http://caixa.com.br.fgtsagendesaqueconta.com/c...
2,http://hissoulreason.com/js/homepage/home/
3,http://unauthorizd.newebpage.com/webapps/66fbf/
4,http://133.130.103.10/23/


In [3]:
len(computer_raw_data)

996

In [4]:
type(computer_raw_data)

pandas.core.frame.DataFrame

# Feature Extraction

Features Extarcted are:
    
Having IP: If the domain path is having IP Address or not.
    
Long url: Length of url.
    
have_at_symbol: Having @ symbol or not.
    
redirection:If the url has symbol(//) after protocol then such URL is to be classified as phishing.
    
prefix_suffix_separation:If the domain has '-' symbol then it is considered as phishing site.

sub_domains:If the url has more than 3 dots then it is a phishing.
    
shortening_service:If the url is tiny phishing otherwise legitimate.
    
and the other fearures are google_index,abnormal_url,web_traffic,domain_registration_length,age_domain,dns_record,statistical_report,https_token.







In [5]:
cd = None
class FeatureExtraction:
    def __init__(self):
        pass
    
    def getProtocol(self,url):
        return urlparse(url).scheme
    
    def getDomain(self,url):
        return urlparse(url).netloc
    
    def getPath(self,url):
        return urlparse(url).path
    
    def havingIP(self,url):
        """If the domain part has IP then it is phishing otherwise legitimate"""
        match=re.search('(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  #IPv4
                    '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  #IPv4 in hexadecimal
                    '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}',url)     #Ipv6
        if match:
            #print match.group()
            return 1            # phishing
        else:
            #print 'No matching pattern found'
            return 0            # legitimate
    
    def long_url(self,url):
        """This function is defined in order to differntiate website based on the length of the URL"""
        if len(url) < 54:
            return 0            # legitimate
        elif len(url) >= 54 and len(url) <= 75:
            return 2            # suspicious
        else:
            return 1            # phishing
    
    def have_at_symbol(self,url):
        """This function is used to check whether the URL contains @ symbol or not"""
        if "@" in url:
            return 1            # phishing
        else:
            return 0            # legitimate
    
    def redirection(self,url):
        """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
        if "//" in urlparse(url).path:
            return 1            # phishing
        else:
            return 0            # legitimate
        
    def prefix_suffix_separation(self,url):
        """If the domain has '-' symbol then it is considered as phishing site"""
        if "-" in urlparse(url).netloc:
            return 1            # phishing
        else:
            return 0            # legitimate
        
    def sub_domains(self,url):
        """If the url has more than 3 dots then it is a phishing"""
        if url.count(".") < 3:
            return 0              # legitimate        
        elif url.count(".") == 3:
            return 2  
        else:
            return 1            # phishing
        
    def shortening_service(self,url):
        """Tiny URL -> phishing otherwise legitimate"""
        match=re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net',url)
        if match:
            return 1               # phishing
        else:
            return 0               # legitimate
        
    
    def google_index(self,url):
        user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'
        headers = { 'User-Agent' : user_agent}
        query = {'q': 'info:' + url}
        google = "https://www.google.com/search?" + urlencode(query)
        #data = requests.get(google, headers=headers,proxies=proxies)
        data = requests.get(google,headers=headers)
        data.encoding = 'ISO-8859-1'
        soup = BeautifulSoup(str(data.content), "html.parser")
        try:
            check = soup.find(id="rso").find("div").find("div").find("h3").find("a")
            if soup.find(id="rso").find("div").find("div").find("h3").find("a").find("href" != None):
                href = check['href']
                return 0 # indexed
            else:
                return 1
        except AttributeError:
            return 1 # indexed
        #print("Waiting " + str(seconds) + " seconds until checking next URL.\n")
        #time.sleep(float(seconds))
    
    
    def abnormal_url(self,url):
        dns = 0
        #domain_name = ""
        try:
            #domain = urlparse(url).netloc
            #print(domain)
            domain_name = whois.whois(urlparse(url).netloc)
            #print(domain_name)
        except:
            dns = 1
        
        if dns == 1:
            return 1 # phishing
        else:
            hostname=domain_name.domain_name
            #match=re.search(hostname,url)
            if hostname in url:
                return 0 # legitimate
            else:
                return 1 # phishing
    
    
    def web_traffic(self,url):
        try:
            rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
        except TypeError:
            return 1
        except HTTPError:
            return 2
        rank= int(rank)
        if (rank<100000):
            return 0
        else:
            return 2
        
    def domain_registration_length(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
        except:
            dns = 1
        
        if dns == 1:
            return 1      #phishing
        else:
            expiration_date = domain_name.expiration_date
            today = time.strftime('%Y-%m-%d')
            today = datetime.strptime(today, '%Y-%m-%d')
            if expiration_date is None:
                return 1
            elif type(expiration_date) is list or type(today) is list :
                return 2     #If it is a type of list then we can't select a single value from list. So,it is regarded as suspected website  
            else:
                creation_date = domain_name.creation_date
                expiration_date = domain_name.expiration_date
                if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
                    try:
                        creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                        expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
                    except:
                        return 2
                registration_length = abs((expiration_date - today).days)
                if registration_length / 365 <= 1:
                    return 1 #phishing
                else:
                    return 0 # legitimate
            
    def age_domain(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
        except:
            dns = 1
        
        if dns == 1:
            return 1
        else:
            creation_date = domain_name.creation_date
            expiration_date = domain_name.expiration_date
            if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
                try:
                    creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                    expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
                except:
                    return 2
            if ((expiration_date is None) or (creation_date is None)):
                return 1
            elif ((type(expiration_date) is list) or (type(creation_date) is list)):
                return 2
            else:
                ageofdomain = abs((expiration_date - creation_date).days)
                if ((ageofdomain/30) < 6):
                    return 1
                else:
                    return 0
     
    
    def dns_record(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
            #rint(domain_name)
        except:
            dns = 1
        
        if dns == 1:
            return 1
        else:
            return 0
        
   
    def statistical_report(self,url):
        hostname = url
        h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
        z = int(len(h))
        if z != 0:
            y = h[0][1]
            hostname = hostname[y:]
            h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
            z = int(len(h))
            if z != 0:
                hostname = hostname[:h[0][0]]
        url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
        try:
            ip_address = socket.gethostbyname(hostname)
            ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42',ip_address)  
        except:
            return 1

        if url_match:
            return 1
        else:
            return 0
        
    def https_token(self,url):
        match=re.search('https://|http://',url)
        try:
            if match.start(0)==0 and match.start(0) is not None:
                url=url[match.end(0):]
                match=re.search('http|https',url)
                if match:
                    return 1
                else:
                    return 0
        except:
            return 1




Declaring Variables

In [6]:
#features
protocol = []
domain = []
path = []
having_ip = []
len_url = []
having_at_symbol = []
redirection_symbol = []
prefix_suffix_separation = []
sub_domains = []
tiny_url = []
abnormal_url = []
web_traffic = []
domain_registration_length = []
dns_record = []
statistical_report = []
age_domain = []
http_tokens = []
google_index = []

### 0 - legitimate
### 1 - phishing
### 2 - suspicious

Checking urls and appending to variables above

In [7]:
# object creation
fe = FeatureExtraction()
rows = len(computer_raw_data["urls"])

for i in range(0,rows):
    url=computer_raw_data["urls"][i]
    protocol.append(fe.getProtocol(url))
    path.append(fe.getPath(url))
    domain.append(fe.getDomain(url))
    having_ip.append(fe.havingIP(url))
    len_url.append(fe.long_url(url))
    having_at_symbol.append(fe.have_at_symbol(url))
    redirection_symbol.append(fe.redirection(url))
    prefix_suffix_separation.append(fe.prefix_suffix_separation(url))
    sub_domains.append(fe.sub_domains(url))
    tiny_url.append(fe.shortening_service(url))
    web_traffic.append(fe.web_traffic(url))
    domain_registration_length.append(fe.domain_registration_length(url))
    dns_record.append(fe.dns_record(url))
    statistical_report.append(fe.statistical_report(url))
    age_domain.append(fe.age_domain(url))
    http_tokens.append(fe.https_token(url))
    google_index.append(fe.google_index(url))
    abnormal_url.append(fe.abnormal_url(url))


Declaring the Phishing website label as 1

In [8]:
label = []
for i in range(0,rows):
    label.append(1)

Forming the Dataframe of Phishing Websites

In [9]:
d={'Protocol':pd.Series(protocol),'Domain':pd.Series(domain),'Path':pd.Series(path),'Having_IP':pd.Series(having_ip),
   'URL_Length':pd.Series(len_url),'Having_@_symbol':pd.Series(having_at_symbol),
   'Redirection_//_symbol':pd.Series(redirection_symbol),'Prefix_suffix_separation':pd.Series(prefix_suffix_separation),
   'Sub_domains':pd.Series(sub_domains),'tiny_url':pd.Series(tiny_url),'web_traffic' : pd.Series(web_traffic) ,
   'domain_registration_length':pd.Series(domain_registration_length),'dns_record':pd.Series(dns_record),
   'statistical_report':pd.Series(statistical_report),'age_domain':pd.Series(age_domain),'http_tokens':pd.Series(http_tokens),'google_index':pd.Series(google_index),'abnormal_url':pd.Series(abnormal_url),
   'label':pd.Series(label)}
data1=pd.DataFrame(d)
data1

,Protocol,Domain,Path,Having_IP,URL_Length,Having_@_symbol,Redirection_//_symbol,Prefix_suffix_separation,Sub_domains,tiny_url,web_traffic,domain_registration_length,dns_record,statistical_report,age_domain,http_tokens,google_index,abnormal_url,label
0,http,asesoresvelfit.com,/media/datacredito.co/,0,0,0,0,0,0,1,1,1,1,0,1,0,1,1,1
1,http,caixa.com.br.fgtsagendesaqueconta.com,/consulta8523211/principal.php,0,1,0,0,0,1,0,1,1,1,1,1,0,1,1,1
2,http,hissoulreason.com,/js/homepage/home/,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,1
3,http,unauthorizd.newebpage.com,/webapps/66fbf/,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,1
4,http,133.130.103.10,/23/,1,0,0,0,0,2,0,1,1,1,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,http,www.guzheng.com.my,/libraries/joomla/session/storage/regional/2f8...,0,1,0,0,0,1,0,1,1,1,0,1,0,1,1,1
992,http,www.ioam.org.cn,/plugins/system/AtendimentoCliente-98.890.332_...,0,1,0,0,0,1,0,1,1,1,0,1,0,1,1,1
993,https,theparlourbeautyrooms.ie,/dehl/Yah/T/Y1.html,0,0,0,0,0,0,0,1,1,1,0,1,0,1,1,1
994,http,loogin-facebook-settinggst.esy.es,/confirmations/,0,2,0,0,1,0,0,1,1,1,1,1,0,1,1,1


In [10]:

data1

,Protocol,Domain,Path,Having_IP,URL_Length,Having_@_symbol,Redirection_//_symbol,Prefix_suffix_separation,Sub_domains,tiny_url,web_traffic,domain_registration_length,dns_record,statistical_report,age_domain,http_tokens,google_index,abnormal_url,label
0,http,asesoresvelfit.com,/media/datacredito.co/,0,0,0,0,0,0,1,1,1,1,0,1,0,1,1,1
1,http,caixa.com.br.fgtsagendesaqueconta.com,/consulta8523211/principal.php,0,1,0,0,0,1,0,1,1,1,1,1,0,1,1,1
2,http,hissoulreason.com,/js/homepage/home/,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,1
3,http,unauthorizd.newebpage.com,/webapps/66fbf/,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,1
4,http,133.130.103.10,/23/,1,0,0,0,0,2,0,1,1,1,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,http,www.guzheng.com.my,/libraries/joomla/session/storage/regional/2f8...,0,1,0,0,0,1,0,1,1,1,0,1,0,1,1,1
992,http,www.ioam.org.cn,/plugins/system/AtendimentoCliente-98.890.332_...,0,1,0,0,0,1,0,1,1,1,0,1,0,1,1,1
993,https,theparlourbeautyrooms.ie,/dehl/Yah/T/Y1.html,0,0,0,0,0,0,0,1,1,1,0,1,0,1,1,1
994,http,loogin-facebook-settinggst.esy.es,/confirmations/,0,2,0,0,1,0,0,1,1,1,1,1,0,1,1,1


Reading all the url's of legitimate text file

In [11]:
raw_data=pd.read_csv("C:\\Users\\gouru\\Downloads\\Phishing-Website-Detection-master\\Phishing-Website-Detection-master\\raw_datasets\\legitimate_urls.txt",header=None,names=['urls'])
raw_data.head()

,urls
0,http://www.liquidgeneration.com/
1,http://www.onlineanime.org/
2,http://www.ceres.dti.ne.jp/~nekoi/senno/senfir...
3,http://www.galeon.com/kmh/
4,http://www.fanworkrecs.com/


In [12]:
len(raw_data["urls"])

1017

Extarcting Features from Legitimate url's usiong above functions 

In [13]:
#features
Protocol = []
Domain = []
Path = []
Having_ip = []
Len_url = []
Having_at_symbol = []
Redirection_symbol = []
Prefix_suffix_separation = []
Sub_domains = []
Tiny_url = []
Abnormal_url = []
Web_traffic = []
Domain_registration_length = []
Dns_record = []
Statistical_report = []
Age_domain = []
Http_tokens = []
Google_index = []

In [14]:
fe = FeatureExtraction()
rows = len(raw_data["urls"])

for i in range(0,rows):
    url=raw_data["urls"][i]
    Protocol.append(fe.getProtocol(url))
    Path.append(fe.getPath(url))
    Domain.append(fe.getDomain(url))
    Having_ip.append(fe.havingIP(url))
    Len_url.append(fe.long_url(url))
    Having_at_symbol.append(fe.have_at_symbol(url))
    Redirection_symbol.append(fe.redirection(url))
    Prefix_suffix_separation.append(fe.prefix_suffix_separation(url))
    Sub_domains.append(fe.sub_domains(url))
    Tiny_url.append(fe.shortening_service(url))
    Web_traffic.append(fe.web_traffic(url))
    Domain_registration_length.append(fe.domain_registration_length(url))
    Dns_record.append(fe.dns_record(url))
    Statistical_report.append(fe.statistical_report(url))
    Age_domain.append(fe.age_domain(url))
    Http_tokens.append(fe.https_token(url))
    Google_index.append(fe.google_index(url))
    Abnormal_url.append(fe.abnormal_url(url))

Declaring Legitimate websites label as 0

In [15]:
Label = []
for i in range(0,rows):
    Label.append(0)

Forming the Legitimate website dataframe

In [16]:
D={'Protocol':pd.Series(Protocol),'Domain':pd.Series(Domain),'Path':pd.Series(Path),'Having_IP':pd.Series(Having_ip),
   'URL_Length':pd.Series(Len_url),'Having_@_symbol':pd.Series(Having_at_symbol),
   'Redirection_//_symbol':pd.Series(Redirection_symbol),'Prefix_suffix_separation':pd.Series(Prefix_suffix_separation),
   'Sub_domains':pd.Series(Sub_domains),'tiny_url':pd.Series(Tiny_url),'web_traffic' : pd.Series(Web_traffic) ,
   'domain_registration_length':pd.Series(Domain_registration_length),'dns_record':pd.Series(Dns_record),
   'statistical_report':pd.Series(Statistical_report),'age_domain':pd.Series(Age_domain),'http_tokens':pd.Series(Http_tokens),'google_index':pd.Series(Google_index),'abnormal_url':pd.Series(Abnormal_url),
   'label':pd.Series(Label)}

  

In [17]:
data2=pd.DataFrame(D)
data2

,Protocol,Domain,Path,Having_IP,URL_Length,Having_@_symbol,Redirection_//_symbol,Prefix_suffix_separation,Sub_domains,tiny_url,web_traffic,domain_registration_length,dns_record,statistical_report,age_domain,http_tokens,google_index,abnormal_url,label
0,http,www.liquidgeneration.com,/,0,0,0,0,0,0,0,2,1,1,0,1,0,1,1,0
1,http,www.onlineanime.org,/,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,0
2,http,www.ceres.dti.ne.jp,/~nekoi/senno/senfirst.html,0,0,0,0,0,1,0,0,1,1,0,1,0,1,1,0
3,http,www.galeon.com,/kmh/,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0
4,http,www.fanworkrecs.com,/,0,0,0,0,0,0,0,2,1,1,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,http,www.penguins.cl,/,0,0,0,0,0,0,0,2,1,1,0,1,0,1,1,0
1013,http,www.falklands.net,/,0,0,0,0,0,0,0,2,1,1,0,1,0,1,1,0
1014,http,englishnf.pinguins.info,/,0,0,0,0,0,0,0,2,1,1,1,1,0,1,1,0
1015,http,www.gdargaud.net,/antarctica/penguins.html,0,0,0,0,0,2,0,2,1,1,0,1,0,1,1,0


In [18]:
type(data2)

pandas.core.frame.DataFrame

Splitting the train and test data foe both Phishing and Legitimate Datframes 

In [19]:
from sklearn.model_selection import train_test_split
y1=data1['label']
x1=data1.drop('label',axis=1)

In [20]:
x1_train,x1_test,y1_train,y1_test=train_test_split(x1,y1,test_size=0.3, random_state=100)

In [21]:
y2=data2['label']
x2=data2.drop('label',axis=1)

In [22]:
x2_train,x2_test,y2_train,y2_test=train_test_split(x2,y2,test_size=0.3, random_state=100)

Concatinating Train and Test data obtained from Phishing and Legitimate Dataframes

In [23]:
x_train=pd.concat([x1_train,x2_train])

In [24]:
x_train

,Protocol,Domain,Path,Having_IP,URL_Length,Having_@_symbol,Redirection_//_symbol,Prefix_suffix_separation,Sub_domains,tiny_url,web_traffic,domain_registration_length,dns_record,statistical_report,age_domain,http_tokens,google_index,abnormal_url
218,http,scs-network.com,/kem/aba.html,0,0,0,0,1,0,0,1,1,1,0,1,0,1,1
360,http,fdhdfhdfhhfdfdhhdfhhfddfdf.blogspot.com,/,0,0,0,0,0,0,1,1,1,1,0,1,0,1,1
543,http,www.caixa.gov.br-fgts.gov.br.trabalhador.servi...,/oferta.cliente,0,1,0,0,1,1,0,1,1,1,1,1,0,1,1
111,http,www.aharsntenetsrelsioncustomizedia.com,/domain/indexx.htm,0,2,0,0,0,2,0,1,1,1,1,1,0,1,1
402,http,inhomecareidaho.com,/wp-includes/news/dropbaka/07b4089c45eabdd3349...,0,1,0,0,0,0,0,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,http,animaldiversity.ummz.umich.edu,/site/accounts/information/sula_nebouxii.html,0,1,0,0,0,1,0,0,1,1,0,1,0,1,1
871,http,www.stanford.edu,/group/stanfordbirds/text/essays/flamingo_feed...,0,1,0,0,0,2,0,0,1,1,0,1,0,1,1
835,http,www.birdlife.org,/datazone/species/index.html,0,1,0,0,0,1,0,2,1,1,0,1,0,1,1
792,http,www.apprenticeschool.com,,0,0,0,0,0,0,0,1,1,1,0,1,0,1,1


In [25]:
y_train=pd.concat([y1_train,y2_train])
y_train

218    1
360    1
543    1
111    1
402    1
      ..
855    0
871    0
835    0
792    0
520    0
Name: label, Length: 1408, dtype: int64

In [26]:
y_test=pd.concat([y1_test,y2_test])
y_test

391    1
483    1
952    1
606    1
610    1
      ..
433    0
427    0
661    0
278    0
218    0
Name: label, Length: 605, dtype: int64

In [27]:
x_test=pd.concat([x1_test,x2_test])
x_test

,Protocol,Domain,Path,Having_IP,URL_Length,Having_@_symbol,Redirection_//_symbol,Prefix_suffix_separation,Sub_domains,tiny_url,web_traffic,domain_registration_length,dns_record,statistical_report,age_domain,http_tokens,google_index,abnormal_url
391,http,www.riclogauto.96.lt,/login.php,0,0,0,0,0,1,0,0,1,1,1,1,0,1,1
483,http,backup.din-14675.de,/tmp/s.png/,0,2,0,0,1,2,0,1,1,1,0,1,0,1,1
952,http,ow.ly,/2G1V30bdj5z,0,0,0,0,0,0,1,2,1,1,1,1,0,1,1
606,http,platerom.ro,/mygenlvl/products/viewer.php,0,0,0,0,0,0,0,1,1,1,0,1,0,1,1
610,http,liceosanconrado.cl,/836gsg5/casts/,0,0,0,0,0,0,0,2,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,http,ourworld.compuserve.com,/homepages/anithmi/cyclocke.htm,0,2,0,0,0,2,0,2,1,1,1,1,0,1,1
427,http,www.softshape.com,/cham/,0,0,0,0,0,0,0,2,1,1,0,1,0,1,1
661,http,universitycommunications.uvm.edu,/tour/,0,2,0,0,0,0,0,0,1,1,0,1,0,1,1
278,http,www.allvid.com,/,0,0,0,0,0,0,0,1,1,1,0,1,0,1,1


In [28]:
x_train['tiny_url'].unique()

array([0, 1], dtype=int64)

In [29]:
x_train.dtypes

Protocol                      object
Domain                        object
Path                          object
Having_IP                      int64
URL_Length                     int64
Having_@_symbol                int64
Redirection_//_symbol          int64
Prefix_suffix_separation       int64
Sub_domains                    int64
tiny_url                       int64
web_traffic                    int64
domain_registration_length     int64
dns_record                     int64
statistical_report             int64
age_domain                     int64
http_tokens                    int64
google_index                   int64
abnormal_url                   int64
dtype: object

# Preprocessing the Data

Label Encoding

In [30]:
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
x_train['Protocol']= label_encoder.fit_transform(x_train['Protocol']) 
x_train['Domain']= label_encoder.fit_transform(x_train['Domain']) 
x_train['Path']= label_encoder.fit_transform(x_train['Path']) 
x_train.drop(['google_index','abnormal_url'],axis=1)

x_test['Protocol']= label_encoder.fit_transform(x_test['Protocol']) 
x_test['Domain']= label_encoder.fit_transform(x_test['Domain']) 
x_test['Path']= label_encoder.fit_transform(x_test['Path']) 
x_test.drop(columns=['google_index' , 'abnormal_url'],axis=1) 

x_train['Protocol'].unique() 

array([0, 1])

Scaling the Data using standard scalar

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_trains=scaler.transform(x_train)
x_tests=scaler.transform(x_test)

# Machine Learning Models to Predict the Test Data

Decision Tree Classifier

In [32]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_trains,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [33]:
y_pred = model.predict(x_tests)

In [34]:
from sklearn.metrics import roc_auc_score, roc_curve
def get_fpr(y_test, y_pred):
    nbenign = (y_test == 0).sum()
    nfalse = (y_pred[y_test == 0] == 1).sum()
    return nfalse / float(nbenign)

In [42]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
print("Accuracy:")
accuracy_score(y_test,y_pred)



[[177 129]
 [ 32 267]]
Accuracy:


0.7338842975206612

In [43]:

print('FPR:')
get_fpr(y_test, y_pred)

FPR:


0.4215686274509804

Random Forest Classifier

In [46]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=1000,max_depth=10, max_leaf_nodes=10000)
rfc.fit(x_trains,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=10000, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [47]:
y_pred_rfc = rfc.predict(x_tests)

In [49]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm_rfc = confusion_matrix(y_test,y_pred_rfc)
print(cm_rfc)

print("Accuracy:")
accuracy_score(y_test,y_pred_rfc)



[[158 148]
 [  8 291]]
Accuracy:


0.7421487603305785

In [50]:

print('FPR:')
get_fpr(y_test, y_pred_rfc)

FPR:


0.48366013071895425

Logistic Regression

In [52]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_trains, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
y_pred_logreg = logreg.predict(x_tests)

In [54]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm_log = confusion_matrix(y_test,y_pred_logreg)
print(cm_log)
print("Accuracy")
accuracy_score(y_test,y_pred_logreg)



[[200 106]
 [ 25 274]]
Accuracy


0.7834710743801653

In [55]:

print('FPR:')
get_fpr(y_test, y_pred_logreg)

FPR:


0.3464052287581699

K-Nearest Neighbours Classifier

In [57]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=9)
neigh.fit(x_trains, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

In [58]:
y_pred_neigh = neigh.predict(x_tests)

In [59]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm_neigh = confusion_matrix(y_test,y_pred_neigh)
print(cm_neigh)

print("Accuracy")
accuracy_score(y_test,y_pred_neigh)



[[223  83]
 [ 23 276]]
Accuracy


0.824793388429752

In [60]:

print('FPR:')
get_fpr(y_test, y_pred_neigh)

FPR:


0.27124183006535946

Grid Search

In [61]:
# Import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Define the grid of values
penalty = ['l1', 'l2']
C = [0.8, 0.9, 1.0]
tol = [0.01, 0.001 ,0.0001]
max_iter = [100, 150, 200, 250]

# Create a dictionary where tol and max_iter are keys and the lists of their values are the corresponding values
param_grid = dict(penalty=penalty, C=C, tol=tol, max_iter=max_iter)

In [62]:
random_model = RandomizedSearchCV(estimator=logreg, param_distributions=param_grid, cv=5)

# Fit random_model to the data
random_model_result = random_model.fit(x_trains, y_train)

# Summarize results
best_score, best_params = random_model_result.best_score_, random_model_result.best_params_
print("Best score: %.2f using %s" % (best_score*100., best_params))

Best score: 83.81 using {'tol': 0.001, 'penalty': 'l2', 'max_iter': 150, 'C': 0.9}


C:\Users\gouru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\gouru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\gouru\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)


Prediction using Neural Networks

In [63]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import *
from tensorflow.keras import layers

Using TensorFlow backend.


In [64]:
model = Sequential()

model.add(Dense(40, activation='relu',
          kernel_initializer='uniform',input_dim=x_trains.shape[1]))
#model.add(layers.Dropout(0.2))
model.add(Dense(30, activation='relu',
          kernel_initializer='uniform'))
#model.add(layers.Dropout(0.5))
model.add(Dense(1,  activation='sigmoid', 
          kernel_initializer='uniform'))

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [65]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 40)                760       
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1230      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 2,021
Trainable params: 2,021
Non-trainable params: 0
_________________________________________________________________


In [66]:
from keras import callbacks

es_cb = callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=5)

In [67]:
history = model.fit(x_trains, y_train, batch_size=64, epochs=1000, verbose=1, callbacks=[es_cb])

scores = model.evaluate(x_tests, y_test)


Epoch 1/1000
1408/1408 [==============================] - 0s 163us/step - loss: 0.6899 - accuracy: 0.7528
Epoch 2/1000
1408/1408 [==============================] - 0s 17us/step - loss: 0.6637 - accuracy: 0.8381
Epoch 3/1000
1408/1408 [==============================] - 0s 17us/step - loss: 0.5718 - accuracy: 0.8466
Epoch 4/1000
1408/1408 [==============================] - 0s 16us/step - loss: 0.4390 - accuracy: 0.8487
Epoch 5/1000
1408/1408 [==============================] - 0s 18us/step - loss: 0.3667 - accuracy: 0.8473
Epoch 6/1000
1408/1408 [==============================] - 0s 17us/step - loss: 0.3496 - accuracy: 0.8494
Epoch 7/1000
1408/1408 [==============================] - 0s 20us/step - loss: 0.3386 - accuracy: 0.8558
Epoch 8/1000
1408/1408 [==============================] - 0s 18us/step - loss: 0.3293 - accuracy: 0.8587
Epoch 9/1000
1408/1408 [==============================] - 0s 17us/step - loss: 0.3203 - accuracy: 0.8580
Epoch 10/1000
1408/1408 [=============================

# Output:

| Model | Accuracy   |
|------|------------------------------|
| Decision Tree Calssifier   | 74.6|
|
| Random Forest Calssifier   | 75|
|
| Logistic Regression  | 78.4 |
|
|K-Nearest neighbours  | 82.5|
|
|Grid Search | 84|
|
|Neural Network | 93.4|
